# Remake the EDA notebook for the streamlit app

### Librairy imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy as sp
import plotly.express as xp

### Load the cleaned csv file

In [2]:
# Import the csv file
df = pd.read_csv("../Clean_Data/2024/clean_dataset_legislative_2024.csv", sep=";")


C:\Users\Ronan\AppData\Local\Temp\ipykernel_8048\1881465511.py:2: DtypeWarning: Columns (0,2,82,83,84,85,87,88,91,92,93,94,96,97,100,101,102,103,105,106,109,110,111,112,114,115,118,119,120,121,123,124,127,128,129,130,132,133,136,137,138,139,141,142,145,146,147,148,150,151,154,155,156,157,159,160,163,164,165,166,168,169,172,173,174,175,177,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Clean_Data/2024/clean_dataset_legislative_2024.csv", sep=";")


## Number of candidates analysis (Univariate and Multivariate)

In [3]:
count_by_candidates = df.groupby("Nombre_candidats").agg(
    Number_of_Cities= ("Libellé_commune", "size"),
    Average_Inscrits = ("Inscrits", "mean")
).reset_index()
count_by_candidates.head(15)

,Nombre_candidats,Number_of_Cities,Average_Inscrits
0,1,14,3548.857143
1,2,28026,1117.415471
2,3,3132,1901.227011
3,4,174,11015.637931
4,5,22,46795.181818
5,6,8,40157.375000
6,7,1,87289.000000
7,8,8,122088.125000
8,9,1,68957.000000
9,10,1,400366.000000


In [4]:
import plotly.express as px

# Assuming 'count_by_candidates' is a Pandas DataFrame
fig = px.bar(count_by_candidates, 
            x="Nombre_candidats", 
            y="Number_of_Cities",
            title="Number of Cities vs. Nombre de Candidats",
            labels={'Nombre_candidats':'Nombre de Candidats', 
            'Number_of_Cities':'Number of Cities'}
            )
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [5]:
# Most second turn confrontations that happen are either duels or triangles followed by 4 candidates
df_filtered = df[df["Nombre_candidats"] > 7].sort_values(by="Inscrits", ascending=False).reset_index()
df_filtered.head(15)

,index,Code_département,Libellé_département,Code_commune,Libellé_commune,Inscrits,Votants,Pourcentage_Votants,Abstentions,Pourcentage_Abstentions,...,Numéro_de_panneau_18,Nuance_candidat_18,Nom_candidat_18,Prénom_candidat_18,Sexe_candidat_18,Voix_18,Pourcentage_Voix/inscrits_18,Pourcentage_Voix/exprimés_18,Elu_18,Nombre_candidats
0,26004,75,Paris,75056,Paris,687417,459856,"66,90%",227561,"33,10%",...,12.0,UG,ROSSET,Marine,FEMININ,21784.0,"3,17%","5,00%",False,18
1,3167,13,Bouches-du-Rhône,13055,Marseille,400366,258783,"64,64%",141583,"35,36%",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,10
2,24074,69,Rhône,69123,Lyon,225440,166294,"73,76%",59146,"26,24%",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,8
3,10580,31,Haute-Garonne,31555,Toulouse,196238,135652,"69,13%",60586,"30,87%",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,12
4,11133,33,Gironde,33063,Bordeaux,166625,114734,"68,86%",51891,"31,14%",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,8
5,23522,67,Bas-Rhin,67482,Strasbourg,142722,97767,"68,50%",44955,"31,50%",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,8
6,20039,59,Nord,59350,Lille,122500,81459,"66,50%",41041,"33,50%",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,13
7,14964,44,Loire-Atlantique,44109,Nantes,106987,73390,"68,60%",33597,"31,40%",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,12
8,11688,34,Hérault,34172,Montpellier,101456,64608,"63,68%",36848,"36,32%",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,8
9,17221,51,Marne,51454,Reims,101169,59940,"59,25%",41229,"40,75%",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,8


### Add the region column

In [6]:
# Use mapping to add the regions of the departement
departements_regions = {
    "01": "Auvergne-Rhône-Alpes",
    "02": "Hauts-de-France",
    "03": "Auvergne-Rhône-Alpes",
    "04": "Provence-Alpes-Côte d'Azur",
    "05": "Provence-Alpes-Côte d'Azur",
    "06": "Provence-Alpes-Côte d'Azur",
    "07": "Auvergne-Rhône-Alpes",
    "08": "Grand Est",
    "09": "Occitanie",
    "10": "Grand Est",
    "11": "Occitanie",
    "12": "Occitanie",
    "13": "Provence-Alpes-Côte d'Azur",
    "14": "Normandie",
    "15": "Auvergne-Rhône-Alpes",
    "16": "Nouvelle-Aquitaine",
    "17": "Nouvelle-Aquitaine",
    "18": "Centre-Val de Loire",
    "19": "Nouvelle-Aquitaine",
    "2A": "Corse",
    "2B": "Corse",
    "21": "Bourgogne-Franche-Comté",
    "22": "Bretagne",
    "23": "Nouvelle-Aquitaine",
    "24": "Nouvelle-Aquitaine",
    "25": "Bourgogne-Franche-Comté",
    "26": "Auvergne-Rhône-Alpes",
    "27": "Normandie",
    "28": "Centre-Val de Loire",
    "29": "Bretagne",
    "30": "Occitanie",
    "31": "Occitanie",
    "32": "Occitanie",
    "33": "Nouvelle-Aquitaine",
    "34": "Occitanie",
    "35": "Bretagne",
    "36": "Centre-Val de Loire",
    "37": "Centre-Val de Loire",
    "38": "Auvergne-Rhône-Alpes",
    "39": "Bourgogne-Franche-Comté",
    "40": "Nouvelle-Aquitaine",
    "41": "Centre-Val de Loire",
    "42": "Auvergne-Rhône-Alpes",
    "43": "Auvergne-Rhône-Alpes",
    "44": "Pays de la Loire",
    "45": "Centre-Val de Loire",
    "46": "Occitanie",
    "47": "Nouvelle-Aquitaine",
    "48": "Occitanie",
    "49": "Pays de la Loire",
    "50": "Normandie",
    "51": "Grand Est",
    "52": "Grand Est",
    "53": "Pays de la Loire",
    "54": "Grand Est",
    "55": "Grand Est",
    "56": "Bretagne",
    "57": "Grand Est",
    "58": "Bourgogne-Franche-Comté",
    "59": "Hauts-de-France",
    "60": "Hauts-de-France",
    "61": "Normandie",
    "62": "Hauts-de-France",
    "63": "Auvergne-Rhône-Alpes",
    "64": "Nouvelle-Aquitaine",
    "65": "Occitanie",
    "66": "Occitanie",
    "67": "Grand Est",
    "68": "Grand Est",
    "69": "Auvergne-Rhône-Alpes",
    "70": "Bourgogne-Franche-Comté",
    "71": "Bourgogne-Franche-Comté",
    "72": "Pays de la Loire",
    "73": "Auvergne-Rhône-Alpes",
    "74": "Auvergne-Rhône-Alpes",
    "75": "Île-de-France",
    "76": "Normandie",
    "77": "Île-de-France",
    "78": "Île-de-France",
    "79": "Nouvelle-Aquitaine",
    "80": "Hauts-de-France",
    "81": "Occitanie",
    "82": "Occitanie",
    "83": "Provence-Alpes-Côte d'Azur",
    "84": "Provence-Alpes-Côte d'Azur",
    "85": "Pays de la Loire",
    "86": "Nouvelle-Aquitaine",
    "87": "Nouvelle-Aquitaine",
    "88": "Grand Est",
    "89": "Bourgogne-Franche-Comté",
    "90": "Bourgogne-Franche-Comté",
    "91": "Île-de-France",
    "92": "Île-de-France",
    "93": "Île-de-France",
    "94": "Île-de-France",
    "95": "Île-de-France",
    "971": "Outre-Mer",
    "972": "Outre-Mer",
    "973": "Outre-Mer",
    "974": "Outre-Mer",
    "975": "Outre-Mer",
    "976": "Outre-Mer",
    "987": "Outre-Mer",
    "988": "Outre-Mer",
    "ZX": "Outre-Mer",
    "ZZ": "Étranger",  
}

In [7]:
# Adding a "0" value before the one digit numbers
df["Code_département"] = df["Code_département"].astype(str).apply(lambda x: "0" + x if len(x) == 1 else x)
uniques_departements = df["Code_département"].unique()
print(f"List of the uniques_departements: {uniques_departements}")
# Create the new Region column with mapping
df['Libellé_Région'] = df['Code_département'].map(departements_regions)
# Display the unique regions 
unique_regions = df['Libellé_Région'].unique()
print(f"List of the unique_regions: {unique_regions}")

List of the uniques_departements: ['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '2A' '2B' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41'
 '42' '43' '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55'
 '56' '57' '58' '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69'
 '70' '71' '72' '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83'
 '84' '85' '86' '87' '88' '89' '90' '91' '92' '93' '94' '95' '971' '972'
 '973' '974' '975' '976' '987' '988' 'ZX' 'ZZ']
List of the unique_regions: ['Auvergne-Rhône-Alpes' 'Hauts-de-France' "Provence-Alpes-Côte d'Azur"
 'Grand Est' 'Occitanie' 'Normandie' 'Nouvelle-Aquitaine'
 'Centre-Val de Loire' 'Bourgogne-Franche-Comté' 'Bretagne' 'Corse'
 'Pays de la Loire' 'Île-de-France' 'Outre-Mer' 'Étranger']


In [8]:
# Create a new dataset to store the values we wanna proceed in the analysis
df_candidates = pd.concat([
                                df['Nom_candidat_' + str(i)] + ' ' + 
                                df['Prénom_candidat_' + str(i)] + '#' +
                                df['Sexe_candidat_' + str(i)] + '#' +
                                df['Nuance_candidat_' + str(i)] + '#' +
                                df['Elu_' + str(i)].astype(str) + '#' + 
                                df['Libellé_Région'] + '#' +
                                df['Libellé_département'] + '#' +
                                df['Libellé_commune'] 
                                for i in range(1, 19)], ignore_index=True)
# Drop any NaN values
df_candidates.dropna(inplace=True)
# Split the combined data and keep only 'Nom_complet' and 'Sexe'
df_candidates = df_candidates.str.split('#', expand=True)
df_candidates.columns = ['Nom_complet', 'Sexe', 'Nuance', 'Elu', 'Région', 'Département', 'Commune']
df_candidates.head(20)

,Nom_complet,Sexe,Nuance,Elu,Région,Département,Commune
0,COQUELET Christophe,MASCULIN,ENS,False,Auvergne-Rhône-Alpes,Ain,L'Abergement-Clémenciat
1,PISANI Florence,FEMININ,UG,False,Auvergne-Rhône-Alpes,Ain,L'Abergement-de-Varey
2,PISANI Florence,FEMININ,UG,False,Auvergne-Rhône-Alpes,Ain,Ambérieu-en-Bugey
3,COQUELET Christophe,MASCULIN,ENS,False,Auvergne-Rhône-Alpes,Ain,Ambérieux-en-Dombes
4,GIVERNET Olga,FEMININ,ENS,True,Auvergne-Rhône-Alpes,Ain,Ambléon
5,PISANI Florence,FEMININ,UG,False,Auvergne-Rhône-Alpes,Ain,Ambronay
6,KOTARAC Andréa,MASCULIN,RN,False,Auvergne-Rhône-Alpes,Ain,Ambutrix
7,GIVERNET Olga,FEMININ,ENS,True,Auvergne-Rhône-Alpes,Ain,Andert-et-Condon
8,GIVERNET Olga,FEMININ,ENS,True,Auvergne-Rhône-Alpes,Ain,Anglefort
9,PISANI Florence,FEMININ,UG,False,Auvergne-Rhône-Alpes,Ain,Apremont


In [9]:
# Add the tendency of the candidates
# Create a mapping for the tendency of all the parties from Gauche Radicale to Extrême Droite
tendency_mapping = {
    "ENS": "Centre-Droite",
    "UG": "Gauche",
    "RN": "Extrême-Droite",
    "LR": "Droite",
    "UXD": "Extrême-Droite",
    "DVD": "Droite",
    "HOR": "Centre-Droite",
    "DVC": "Centre",
    "REG": "Autres",
    "EXD": "Extrême-Droite",
    "DIV": "Autres",
    "ECO": "Gauche-Radicale",
    "DVG": "Gauche",
    "UDI": "Droite",
    "SOC": "Gauche",
    "DSV": "Extrême-Droite",
    "FI": "Gauche-Radicale"  
}

# Create the new Region column with mapping
df_candidates['Tendency'] = df_candidates['Nuance'].map(tendency_mapping)

In [10]:
# candidates for the city of Marseille
df_marseille = df_candidates[df_candidates['Commune'] == 'Marseille']
df_marseille.head(10).drop('Commune', axis=1).sort_values('Tendency', ascending=False).reset_index(drop=True)

,Nom_complet,Sexe,Nuance,Elu,Région,Département,Tendency
0,LÉCORCHÉ Pascaline,FEMININ,UG,False,Provence-Alpes-Côte d'Azur,Bouches-du-Rhône,Gauche
1,KESSACI Amine,MASCULIN,UG,False,Provence-Alpes-Côte d'Azur,Bouches-du-Rhône,Gauche
2,LHARDIT Laurent,MASCULIN,UG,True,Provence-Alpes-Côte d'Azur,Bouches-du-Rhône,Gauche
3,DAVI Hendrik,MASCULIN,DVG,True,Provence-Alpes-Côte d'Azur,Bouches-du-Rhône,Gauche
4,JUSTE Christine,FEMININ,UG,False,Provence-Alpes-Côte d'Azur,Bouches-du-Rhône,Gauche
5,FAYSSAT Olivier,MASCULIN,UXD,True,Provence-Alpes-Côte d'Azur,Bouches-du-Rhône,Extrême-Droite
6,GRISETI Monique,FEMININ,RN,True,Provence-Alpes-Côte d'Azur,Bouches-du-Rhône,Extrême-Droite
7,RIOULT Olivier,MASCULIN,RN,False,Provence-Alpes-Côte d'Azur,Bouches-du-Rhône,Extrême-Droite
8,LIQUORI Franck,MASCULIN,RN,False,Provence-Alpes-Côte d'Azur,Bouches-du-Rhône,Extrême-Droite
9,LELOUIS Gisèle,FEMININ,RN,True,Provence-Alpes-Côte d'Azur,Bouches-du-Rhône,Extrême-Droite


In [11]:
df_candidates.to_csv("../Clean_Data/2024/candidates.csv", sep=";", header=True, index=False)

In [16]:
df_region_elu = df_candidates[df_candidates['Elu'] == "True"]
df_region_elu.head(10)
        # Filter the DataFrame
new_filtered_department_df = df_region_elu[df_region_elu["Département"] == 'Bouches-du-Rhône']
new_filtered_department_df = new_filtered_department_df.drop_duplicates(subset='Nom_complet', keep="first")
new_filtered_department_df.head(10)

tendencies_plot_order = ['Gauche-Radicale','Gauche', 'Centre', 'Centre-Droite', 'Droite', 'Extrême-Droite', 'Autres']
tendency_count = new_filtered_department_df["Tendency"].value_counts()
print(tendency_count)


Tendency
Extrême-Droite    10
Gauche             3
Name: count, dtype: int64


In [13]:
df_region_elu.head(10)

,Nom_complet,Sexe,Nuance,Elu,Région,Département,Commune,Tendency
4,GIVERNET Olga,FEMININ,ENS,True,Auvergne-Rhône-Alpes,Ain,Ambléon,Centre-Droite
7,GIVERNET Olga,FEMININ,ENS,True,Auvergne-Rhône-Alpes,Ain,Andert-et-Condon,Centre-Droite
8,GIVERNET Olga,FEMININ,ENS,True,Auvergne-Rhône-Alpes,Ain,Anglefort,Centre-Droite
13,GIVERNET Olga,FEMININ,ENS,True,Auvergne-Rhône-Alpes,Ain,Arboys en Bugey,Centre-Droite
29,GIVERNET Olga,FEMININ,ENS,True,Auvergne-Rhône-Alpes,Ain,Valserhône,Centre-Droite
30,GIVERNET Olga,FEMININ,ENS,True,Auvergne-Rhône-Alpes,Ain,Belley,Centre-Droite
39,GIVERNET Olga,FEMININ,ENS,True,Auvergne-Rhône-Alpes,Ain,Billiat,Centre-Droite
51,GIVERNET Olga,FEMININ,ENS,True,Auvergne-Rhône-Alpes,Ain,Brégnier-Cordon,Centre-Droite
53,GIVERNET Olga,FEMININ,ENS,True,Auvergne-Rhône-Alpes,Ain,Brens,Centre-Droite
62,GIVERNET Olga,FEMININ,ENS,True,Auvergne-Rhône-Alpes,Ain,Cessy,Centre-Droite
